In [1]:
import psycopg2
import pandas.io.sql as pd_sql
import pandas as pd

Data is from the [NTSB Aviation Accident Database](https://app.ntsb.gov/avdata/), which is provided in MS Access format. I exported it to a local PostgreSQL database using `mdb-tools`, with the following commands:


(Command line:)
```
mdb-schema avall.mdb postgres | tr 'A-Z' 'a-z' | psql -d avall
mdb-export avall.mdb $tablename > csv/$tablename.csv
```

(In psql:)
```
\copy $tablename from './$tablename.csv' with delimiter ',' csv header;
```

In [2]:
conn = psycopg2.connect("dbname=avall user=aaron")
cur = conn.cursor()

First, let's create a view using unique aircraft in the `findings` table. Here `02` means human error and `None` is no human error. This will serve as the 'master' list.

In [3]:
query = """
CREATE OR REPLACE VIEW master AS
(
    WITH human_error AS
    (
        SELECT
            ev_id,
            aircraft_key,
            category_no
        FROM
            findings
        WHERE
            category_no = '02'
    )
    SELECT DISTINCT
        findings.ev_id,
        findings.aircraft_key,
        human_error.category_no
    FROM
        findings
        LEFT JOIN
            human_error
        ON
            findings.ev_id = human_error.ev_id
            AND findings.aircraft_key = human_error.aircraft_key
);
"""
cur.execute(query)

Now let's get our desired columns from `aircraft`...

In [4]:
query = """
CREATE OR REPLACE VIEW aircraft_selected AS
(
    SELECT
        ev_id,
        aircraft_key,
        cert_max_gr_wt,
        total_seats,
        num_eng,
        afm_hrs_last_insp,
        afm_hrs,
        dprt_time,
        flt_plan_filed,
        damage,
        acft_category,
        homebuilt,
        type_last_insp,
        oper_same,
        certs_held,
        type_fly,
        second_pilot,
        dprt_pt_same_ev
    FROM
        aircraft
);
"""
cur.execute(query)

...from `events`...

In [5]:
query = """
CREATE OR REPLACE VIEW events_selected AS 
(
    SELECT
        ev_id,
        ev_time,
        ev_type,
        apt_dist,
        apt_elev,
        sky_nonceil_ht,
        sky_ceil_ht,
        vis_rvr,
        vis_rvv,
        vis_sm,
        wx_temp,
        wx_dew_pt,
        wind_vel_kts,
        gust_kts,
        altimeter,
        inj_tot_t,
        mid_air,
        on_ground_collision,
        ev_nr_apt_loc,
        light_cond,
        sky_cond_nonceil,
        sky_cond_ceil,
        ev_highest_injury 
    FROM
        events
);
"""
cur.execute(query)

...from `flight_crew`, only for the first two crew members...

In [6]:
query = """
SELECT
    ev_id,
    aircraft_key,
    crew_no,
    crew_category,
    crew_age,
    med_certf,
    med_crtf_vldty,
    seatbelts_used,
    shldr_harn_used,
    crew_tox_perf,
    pc_profession
FROM
    flight_crew
WHERE
    crew_no <= 2
LIMIT 10;
"""
cur.execute(query)

Now merge our `flight_crew` data with `flight_time`, keeping the columns we want, for crew member 1...

In [7]:
query = """
CREATE OR REPLACE VIEW crew_1 AS
(
    WITH xp AS
    (
        SELECT
            ev_id,
            aircraft_key,
            crew_no,
            flight_hours
        FROM
            flight_time
        WHERE
            flight_type = 'TOTL'
            AND flight_craft = 'ALL '
    )
    SELECT
        fc.ev_id,
        fc.aircraft_key,
        fc.crew_no,
        fc.crew_category,
        fc.crew_age,
        fc.med_certf,
        fc.med_crtf_vldty,
        fc.seatbelts_used,
        fc.shldr_harn_used,
        fc.crew_tox_perf,
        fc.pc_profession,
        xp.flight_hours
    FROM
        flight_crew AS fc
    LEFT JOIN xp ON
        fc.ev_id=xp.ev_id
        AND fc.aircraft_key=xp.aircraft_key
        AND fc.crew_no=xp.crew_no
    WHERE
        fc.crew_no = 1
);
"""
cur.execute(query)

...and for crew member 2.

In [8]:
query = """
CREATE OR REPLACE VIEW crew_2 AS
(
    WITH xp AS
    (
        SELECT
            ev_id,
            aircraft_key,
            crew_no,
            flight_hours
        FROM
            flight_time
        WHERE
            flight_type = 'TOTL'
            AND flight_craft = 'ALL '
    )
    SELECT
        fc.ev_id,
        fc.aircraft_key,
        fc.crew_no,
        fc.crew_category,
        fc.crew_age,
        fc.med_certf,
        fc.med_crtf_vldty,
        fc.seatbelts_used,
        fc.shldr_harn_used,
        fc.crew_tox_perf,
        fc.pc_profession,
        xp.flight_hours
    FROM
        flight_crew AS fc
    LEFT JOIN xp ON
        fc.ev_id=xp.ev_id
        AND fc.aircraft_key=xp.aircraft_key
        AND fc.crew_no=xp.crew_no
    WHERE
        fc.crew_no = 2
);
"""
cur.execute(query)

Now sew all our views together and use `pd_sql` to convert to a DataFrame.

In [9]:
query = """
SELECT
    master.*,
    aircraft_selected.*,
    events_selected.*,
    crew_1.*,
    crew_2.*
FROM
    master
LEFT JOIN
    aircraft_selected
    ON
        master.ev_id = aircraft_selected.ev_id
        AND master.aircraft_key = aircraft_selected.aircraft_key
LEFT JOIN
    events_selected
    ON
        master.ev_id = events_selected.ev_id
LEFT JOIN
    crew_1
    ON
        master.ev_id = crew_1.ev_id
        AND master.aircraft_key = crew_1.aircraft_key
LEFT JOIN
    crew_2
    ON
        master.ev_id = crew_2.ev_id
        AND master.aircraft_key = crew_2.aircraft_key
;
"""
adf = pd_sql.read_sql(query, conn)

Pickle our df, and let's get a CSV in there too for more portable storage.

In [10]:
adf.to_pickle('./adf_raw.pkl')
adf.to_csv('./adf_raw.csv')